In [9]:
import pandas as pd
import csv
import ast

def load_safe_csv(path: str) -> pd.DataFrame:
    """
    Safely loads a pediatric CSV file where 'query' may contain commas.
    Ensures exactly 10 output columns.
    """
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)
        rows = []

        for row in reader:
            # Merge everything between 'query' and 'label'
            if len(row) > 10:
                fixed = row[:1] + [",".join(row[1:-8])] + row[-8:]
                rows.append(fixed)
            else:
                rows.append(row)

    df = pd.DataFrame(rows, columns=header[:10])
    return df


def safe_parse_list(val):
    """Safely parse a string into a Python list, even if malformed."""
    if not isinstance(val, str):
        return []
    val = val.strip()
    if not val:
        return []
    if val.count('[') > val.count(']'):
        val = val + ']'
    try:
        parsed = ast.literal_eval(val)
        return parsed if isinstance(parsed, list) else []
    except Exception:
        # fallback: handle manually split comma-separated values
        if ',' in val:
            return [v.strip().strip("'").strip('"') for v in val.strip('[]').split(',')]
        return [val.strip("[]'\"")]


# --- Load both files safely ---
p1 = load_safe_csv("pediatrics1.csv")
p2 = load_safe_csv("pediatrics2.csv")

print(f"✅ Loaded safely:\n p1 shape: {p1.shape}\n p2 shape: {p2.shape}")

# --- Standardize columns ---
p1.columns = p2.columns = [
    "row", "query", "label", "complexity", "recommended_llm",
    "needs_followup", "distractor_label", "biomarkers",
    "is_high_risk", "query_length"
]

# --- Parse biomarkers safely ---
p1["biomarkers"] = p1["biomarkers"].apply(safe_parse_list)
p2["biomarkers"] = p2["biomarkers"].apply(safe_parse_list)

# --- Combine both ---
combined = pd.concat([p1, p2], ignore_index=True)
combined.drop(columns=["row"], inplace=True, errors="ignore")

print(f"✅ Combined shape: {combined.shape}")

# --- Save clean file ---
combined.to_csv("pediatrics_combined.csv", index=False)
print("💾 Saved cleaned combined file as pediatrics_combined.csv")

# --- Summary ---
biomarker_counts = combined["biomarkers"].explode().value_counts()
print("\n🔬 Top 10 biomarkers:")
print(biomarker_counts.head(10))

print(f"\nTotal unique biomarkers: {biomarker_counts.shape[0]}")
print(f"Rows with empty biomarkers: {(combined['biomarkers'].apply(len) == 0).sum()}")


✅ Loaded safely:
 p1 shape: (500, 10)
 p2 shape: (600, 10)
✅ Combined shape: (1100, 9)
💾 Saved cleaned combined file as pediatrics_combined.csv

🔬 Top 10 biomarkers:
biomarkers
hba1c           33
carbohydrate    33
protein         30
iron            28
glucose         27
vitamin_d       27
weight          27
phosphate       24
potassium       21
zinc            19
Name: count, dtype: int64

Total unique biomarkers: 66
Rows with empty biomarkers: 544


In [3]:
df =pd.read_csv("pediatrics_combined.csv")

In [4]:
print(df.head(10))

                                               query           label  \
0  Medical nutrition therapy for a 6-month-old pr...         therapy   
1  Recommend high-energy weaning foods for a 9-mo...  recommendation   
2  Compare energy density of fortified akamu vs c...      comparison   
3  How does catch-up growth affect protein needs ...         general   
4  Compare iron absorption from fortified formula...      comparison   
5  Nutrition support for a 4-month-old ex-28wk pr...         therapy   
6  Recommend appropriate portion sizes for solid ...  recommendation   
7  Compare zinc content in breast milk vs fortifi...      comparison   
8  How does intrauterine growth restriction modif...         general   
9  Medical nutrition therapy for a 5-month-old wi...         therapy   

   complexity recommended_llm  needs_followup distractor_label   biomarkers  \
0           5         Mistral           False   recommendation           []   
1           4         Mistral            True    

In [6]:
df.head(10)


,query,label,complexity,recommended_llm,needs_followup,distractor_label,biomarkers,is_high_risk,query_length
0,Medical nutrition therapy for a 6-month-old pr...,therapy,5,Mistral,False,recommendation,[],True,16
1,Recommend high-energy weaning foods for a 9-mo...,recommendation,4,Mistral,True,therapy,[],True,12
2,Compare energy density of fortified akamu vs c...,comparison,4,Mistral,True,recommendation,[],False,14
3,How does catch-up growth affect protein needs ...,general,3,Mistral,False,therapy,['protein'],False,10
4,Compare iron absorption from fortified formula...,comparison,4,Mistral,True,recommendation,['iron'],False,14
5,Nutrition support for a 4-month-old ex-28wk pr...,therapy,5,Mistral,False,recommendation,[],True,13
6,Recommend appropriate portion sizes for solid ...,recommendation,3,TinyLlama,True,therapy,[],False,13
7,Compare zinc content in breast milk vs fortifi...,comparison,3,TinyLlama,True,recommendation,['zinc'],False,11
8,How does intrauterine growth restriction modif...,general,4,Mistral,False,therapy,[],True,9
9,Medical nutrition therapy for a 5-month-old wi...,therapy,5,Mistral,False,recommendation,[],True,13


In [10]:
import pandas as pd
import csv
import ast

# ----------------------------
# 1. Safe loader for pediatric files
# ----------------------------
def load_safe_csv(path: str) -> pd.DataFrame:
    """
    Safely loads a pediatric CSV file where 'query' may contain commas.
    Ensures exactly 10 output columns.
    """
    with open(path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)
        rows = []

        for row in reader:
            # Merge everything between 'query' and 'label'
            if len(row) > 10:
                fixed = row[:1] + [",".join(row[1:-8])] + row[-8:]
                rows.append(fixed)
            else:
                rows.append(row)

    df = pd.DataFrame(rows, columns=header[:10])
    return df


def safe_parse_list(val):
    """Safely parse a string into a Python list, even if malformed."""
    if not isinstance(val, str):
        return []
    val = val.strip()
    if not val:
        return []
    if val.count('[') > val.count(']'):
        val = val + ']'
    try:
        parsed = ast.literal_eval(val)
        return parsed if isinstance(parsed, list) else []
    except Exception:
        # fallback: handle manually split comma-separated values
        if ',' in val:
            return [v.strip().strip("'").strip('"') for v in val.strip('[]').split(',')]
        return [val.strip("[]'\"")]

# ----------------------------
# 2. Load pediatric datasets
# ----------------------------
p1 = load_safe_csv("pediatrics1.csv")
p2 = load_safe_csv("pediatrics2.csv")

print(f"✅ Loaded safely:\n p1 shape: {p1.shape}\n p2 shape: {p2.shape}")

# Standardize columns
p1.columns = p2.columns = [
    "row", "query", "label", "complexity", "recommended_llm",
    "needs_followup", "distractor_label", "biomarkers",
    "is_high_risk", "query_length"
]

# Parse biomarkers
p1["biomarkers"] = p1["biomarkers"].apply(safe_parse_list)
p2["biomarkers"] = p2["biomarkers"].apply(safe_parse_list)

# Combine pediatric datasets
pediatrics_combined = pd.concat([p1, p2], ignore_index=True)
pediatrics_combined.drop(columns=["row"], inplace=True, errors="ignore")

print(f"✅ Combined pediatrics shape: {pediatrics_combined.shape}")

# ----------------------------
# 3. Load main dataset for phase 2
# ----------------------------
DATA_PATH = "../data/nutrition_queries_cleaned_for_phase2.csv"
main_df = pd.read_csv(DATA_PATH)

print(f"📦 Main dataset shape: {main_df.shape}")

# ----------------------------
# 4. Align columns before concatenation
# ----------------------------
# Ensure the pediatric dataset has all the same columns as the main one
missing_cols = set(main_df.columns) - set(pediatrics_combined.columns)
for col in missing_cols:
    pediatrics_combined[col] = None

# Reorder columns to match main dataset
pediatrics_combined = pediatrics_combined[main_df.columns]

# ----------------------------
# 5. Concatenate both datasets
# ----------------------------
final_df = pd.concat([main_df, pediatrics_combined], ignore_index=True)

print(f"✅ Final combined dataset shape: {final_df.shape}")

# ----------------------------
# 6. Save final merged dataset
# ----------------------------
output_path = "../data/nutrition_queries_with_pediatrics.csv"
final_df.to_csv(output_path, index=False)
print(f"💾 Saved final merged dataset as {output_path}")

# ----------------------------
# 7. Summary
# ----------------------------
biomarker_counts = final_df["biomarkers"].apply(
    lambda x: safe_parse_list(x)
).explode().value_counts()

print("\n🔬 Top 10 biomarkers:")
print(biomarker_counts.head(10))

print(f"\nTotal unique biomarkers: {biomarker_counts.shape[0]}")
print(f"Rows with empty biomarkers: {(final_df['biomarkers'].apply(len) == 0).sum()}")


✅ Loaded safely:
 p1 shape: (500, 10)
 p2 shape: (600, 10)
✅ Combined pediatrics shape: (1100, 9)
📦 Main dataset shape: (4100, 9)
✅ Final combined dataset shape: (5200, 9)
💾 Saved final merged dataset as ../data/nutrition_queries_with_pediatrics.csv

🔬 Top 10 biomarkers:
biomarkers
calcium      274
iron         162
ferritin     156
oxalate      149
uric_acid    149
vitamin_d    132
glucose      127
hba1c        126
potassium     21
zinc          17
Name: count, dtype: int64

Total unique biomarkers: 67
Rows with empty biomarkers: 544


In [ ]:
d